<a href="https://colab.research.google.com/github/yuriao/DataScienceProjects/blob/main/commonlit_multiBERTmodel_FeatureEngineering_lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This notebook is based on the notebook https://www.kaggle.com/code/ao9mame/commonlit-deberta-with-transformers/notebook with some changes. I have tried to keep the notebook simple baseline short on training time. Have used a transformer model distilroberta-base.

This notebook is also based on:
- https://www.kaggle.com/code/synful/simple-distilroberta-base-10mins-to-train
- https://towardsdatascience.com/how-to-apply-transformers-to-any-length-of-text-a5601410af7f



In [ ]:
from google.colab import drive
drive.mount('commonLit_data')

Drive already mounted at commonLit_data; to attempt to forcibly remount, call drive.mount("commonLit_data", force_remount=True).


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install autocorrect pyspellchecker

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only '../input/' directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os

In [ ]:
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import Dataset as Dataset1
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import torch
import gc
from transformers import TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from autocorrect import Speller
from spellchecker import SpellChecker

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter

import spacy

import lightgbm as lgb

import warnings
warnings.simplefilter('ignore')

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## 1. load data

In [ ]:
DATA_DIR = '/content/commonLit_data/MyDrive/commonLit_data/'

prompts_train = pd.read_csv(DATA_DIR + 'prompts_train.csv')
prompts_test = pd.read_csv(DATA_DIR + 'prompts_test.csv')
summaries_train = pd.read_csv(DATA_DIR + 'summaries_train.csv')
summaries_test = pd.read_csv(DATA_DIR + 'summaries_test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')


## 2. feature engineering function

In [ ]:
class Feature_Engineering:
    def __init__(self) -> None:
        self.STOP_WORDS = set(stopwords.words('english'))
        self.speller = Speller(lang='en')
        self.spellchecker = SpellChecker()
        self.spacy_ner_model = spacy.load('en_core_web_sm')

    def word_overlap_count(self, row):
        ''' intersection(prompt_text, text) '''
        def check_is_stop_word(word):
            return word in self.STOP_WORDS

        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))

    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [' '.join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int) -> int:
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)
        return len(common_ngrams)

    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r"([^']*)", summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):

        wordlist=text.split()
        amount_miss = len(list(self.spellchecker.unknown(wordlist)))

        return amount_miss

    def add_spelling_dictionary(self, tokens):
        '''dictionary update for pyspell checker and autocorrect'''
        self.spellchecker.word_frequency.load_words(tokens)
        self.speller.nlp_data.update({token:1000 for token in tokens})

    def run(self, prompts: pd.DataFrame,summaries:pd.DataFrame) -> pd.DataFrame:

        # before merge preprocess
        prompts['prompt_length'] = prompts['prompt_text'].apply(lambda x: len(word_tokenize(x)))
        prompts['prompt_tokens'] = prompts['prompt_text'].apply(lambda x: word_tokenize(x))

        summaries['summary_length'] = summaries['text'].apply(lambda x: len(word_tokenize(x)))
        summaries['summary_tokens'] = summaries['text'].apply(lambda x: word_tokenize(x))

        # Add prompt tokens into spelling checker dictionary
        prompts['prompt_tokens'].apply(lambda x: self.add_spelling_dictionary(x))

#         from IPython.core.debugger import Pdb; Pdb().set_trace()
        # fix misspelling
        summaries['fixed_summary_text'] = summaries['text'].apply(lambda x: self.speller(x))

        # count misspelling
        summaries['splling_err_num'] = summaries['text'].apply(self.spelling)

        # merge prompts and summaries
        input_df = summaries.merge(prompts, how='left', on='prompt_id')

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']

        input_df['word_overlap_count'] = input_df.apply(self.word_overlap_count, axis=1)

        input_df['bigram_overlap_count'] = input_df.apply(self.ngram_co_occurrence,args=(2,), axis=1)

        input_df['bigram_overlap_ratio'] = input_df['bigram_overlap_count'] / (input_df['summary_length'] - 1)

        input_df['trigram_overlap_count'] = input_df.apply(self.ngram_co_occurrence, args=(3,), axis=1)

        input_df['trigram_overlap_ratio'] = input_df['trigram_overlap_count'] / (input_df['summary_length'] - 2)

        input_df['quotes_count'] = input_df.apply(self.quotes_count, axis=1)

        return input_df.drop(columns=['summary_tokens', 'prompt_tokens'])


In [ ]:
preprocessor = Feature_Engineering()

In [ ]:
#summaries_train['text'] = summaries_train['text'].replace(re.compile(r'[\n\r\t]'), ' ', regex=True)
#prompts_train['prompt_text'] = prompts_train['prompt_text'].replace(re.compile(r'[\n\r\t]'), ' ', regex=True)

#summaries_test['text'] = summaries_test['text'].replace(re.compile(r'[\n\r\t]'), ' ', regex=True)
#prompts_test['prompt_text'] = prompts_test['prompt_text'].replace(re.compile(r'[\n\r\t]'), ' ', regex=True)

In [ ]:
# merge prompt and summaries
#summaries_train = summaries_train.merge(prompts_train, how='left', on='prompt_id')
#summaries_test = summaries_test.merge(prompts_test, how='left', on='prompt_id')

In [ ]:
summaries_train = preprocessor.run(prompts_train, summaries_train)
summaries_test = preprocessor.run(prompts_test, summaries_test)


In [ ]:
summaries_train.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'summary_length', 'fixed_summary_text', 'splling_err_num',
       'prompt_question', 'prompt_title', 'prompt_text', 'prompt_length',
       'length_ratio', 'word_overlap_count', 'bigram_overlap_count',
       'bigram_overlap_ratio', 'trigram_overlap_count',
       'trigram_overlap_ratio', 'quotes_count'],
      dtype='object')

Using GPU. Model is initiated as a regression model with 2 labels, content and wording.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

prediction model class

In [ ]:
class PredictionModel:
    def __init__(self,typee) -> None:
        self.typee = typee
        self.model_fold_dir=f"{self.typee}/BERTModel/fold_{fold}"
        self.training_args = TrainingArguments(
          output_dir=self.model_fold_dir,             # saving directory
          per_device_train_batch_size=12,   # training batch sz
          per_device_eval_batch_size=12,    # validation batch sz
          learning_rate=1.5e-5,            # learning rate
          lr_scheduler_type='linear',      # learning changing scheduler
          num_train_epochs=5,              # エポック数
          save_strategy='epoch',           # チェックポイントの保存タイミング
          logging_strategy='epoch',        # ロギングのタイミング
          evaluation_strategy='epoch',     # 検証セットによる評価のタイミング
          load_best_model_at_end=True,     # 訓練後に開発セットで最良のモデルをロード
          metric_for_best_model='rmse',  # 最良のモデルを決定する評価指標
          greater_is_better=False,         # MCRMSEの場合、低い値が良いのでFalseを設定
          fp16=False,                      # 自動混合精度演算の有効化(CPUを利用する場合、Falseを設定)
          report_to='none',                # WandBへの出力
          save_total_limit=1               # 保存するモデル数
      )
      self.test_args = TrainingArguments(
          output_dir=self.model_fold_dir,
          do_train = False,
          do_predict = True,
          per_device_eval_batch_size = 4,
          dataloader_drop_last = False,
      )

    # Competition metric is Mean Columnwise Root Mean Squared Error（MCRMSE).
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        rmse = mean_squared_error(labels, predictions, squared=False)
        return {'rmse': rmse}

    # autoTokenizaion
    def tokenize_function_train(examples):
        sep = tokenizer1.sep_token
        labels = [examples[self.typee]]
        tokenized = tokenizer1(examples['prompt_title']+sep+examples['prompt_question']+sep+examples['fixed_summary_text'],padding=True,truncation=True,max_length=512)
        return {**tokenized,'labels': labels}

    def tokenize_function_test(examples):
        sep = tokenizer1.sep_token
        tokenized1 = tokenizer1(examples['prompt_title']+sep+examples['prompt_question']+sep+examples['fixed_summary_text'],padding=True,truncation=True,max_length=512)
        return tokenized1

    def huggingface_model_train(model_name,train_content,n_splits):

        data_collator1 = DataCollatorWithPadding(tokenizer=tokenizer1)

        kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

        for fold, (train_idx, val_idx) in enumerate(kfold.split(train_content)):
            print(f'Fold {fold + 1}')

            # training parameter setting


            # dataframe to dataset obj
            train_dataset_content = Dataset1.from_pandas(train_content.iloc[train_idx,:], preserve_index=False) # content
            val_dataset_content = Dataset1.from_pandas(train_content.iloc[val_idx,:], preserve_index=False) # content

            # Mapping tokenizing function to the datasets
            if typee=='content':
              tokenize_function_train = tokenize_function_train_content
            elif typee=='wording':
              tokenize_function_train = tokenize_function_train_wording

            train_tokenized_datasets_content1 = train_dataset_content.map(tokenize_function_train, batched=False)
            val_tokenized_datasets_content1 = val_dataset_content.map(tokenize_function_train, batched=False)

            # model for each fold
            model = AutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=1,
                problem_type='regression',
            ).to(device)

            trainer = Trainer(
                model=model,
                train_dataset=train_tokenized_datasets_content1,
                eval_dataset=val_tokenized_datasets_content1,
                data_collator=data_collator1,
                args=training_args,
                compute_metrics=compute_metrics,
                tokenizer=tokenizer1
            )

            trainer.train()

            trainer.save_model('best_model')

            # save model
            trainer.save_model(f"{typee}/BERTModel/fold_{fold}")


Define tokenizing function for train and test dataset.

model training utility function

prediction function

In [ ]:
def huggingface_model_predict(content,training_folds):
  print('content shape:' + str(content.shape))
  data_collator1 = DataCollatorWithPadding(tokenizer=tokenizer1)
  test_dataset = Dataset1.from_pandas(content, preserve_index=False)
  test_tokenized_dataset1 = test_dataset.map(tokenize_function_test, batched=False)

  content_list=[]
  wording_list=[]
  for fold in range(0,training_folds):
      model = AutoModelForSequenceClassification.from_pretrained(
          f"content/BERTModel/fold_{fold}",
          num_labels=1,
          problem_type='regression',
      ).to(device)
      trainer = Trainer(
          model=model,
          data_collator=data_collator1,
          args=test_args,
          tokenizer=tokenizer1
      )
      content_list.append(trainer.predict(test_tokenized_dataset1))

  for fold in range(0,training_folds):
      model = AutoModelForSequenceClassification.from_pretrained(
          f"wording/BERTModel/fold_{fold}",
          num_labels=1,
          problem_type='regression',
      ).to(device)

      trainer = Trainer(
          model=model,
          data_collator=data_collator1,
          args=test_args,
          tokenizer=tokenizer1
      )
      wording_list.append(trainer.predict(test_tokenized_dataset1))

  content_pred=np.mean(np.array(content_list).T,axis=1)
  wording_pred=np.mean(np.array(wording_list).T,axis=1)

  return content_pred,wording_pred

LGB train

In [ ]:
def LGB_train(all_trainers_content,all_trainers_wording,train_content,typee):
    for count,trainers in enumerate(all_trainers_content):
        pred=huggingface_model_predict(train_content,trainers)
        train_content['content_pred_'+str(count)]=pred

    for count,trainers in enumerate(all_trainers_wording):
        pred=huggingface_model_predict(train_content,trainers)
        train_content['wording_pred_'+str(count)]=pred

    y_train_cv=train_content[typee]
    X_train_cv=train_content.drop(['fixed_summary_text','prompt_question', 'prompt_title', 'prompt_text','content','wording'],axis=1)

    X_train_cv,X_eval_cv,y_train_cv,y_eval_cv=train_test_split(X_train_cv,y_train_cv,test_size=0.3,random_state=42)

    dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
    dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': 0.001,
        'num_leaves': 31,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'max_depth': 10,
        'lambda_l1': 0.0,
        'lambda_l2': 0.011
    }

    evaluation_results = {}
    model = lgb.train(params,
                      num_boost_round=10000,
                        #categorical_feature = categorical_features,
                      valid_names=['train', 'valid'],
                      train_set=dtrain,
                      valid_sets=dval,
                      callbacks=[
                          lgb.early_stopping(stopping_rounds=30, verbose=True),
                            lgb.log_evaluation(100),
                          lgb.callback.record_evaluation(evaluation_results)
                        ],
                      )
    return model

LGB test

In [ ]:
def LGB_test(test_content,content_pred,wording_pred,content_model,wording_model):
    test_content=test_content.drop(['fixed_summary_text','prompt_question', 'prompt_title', 'prompt_text','content','wording'],axis=1)
    for count in range(0,len(content_pred)):
      test_content['content_pred_'+str(count)]=content_pred[count]
      test_content['wording_pred_'+str(count)]=wording_pred[count]
    return content_model.predict(test_content), wording_model.predict(test_content)

New train/validation/test made with just the required columns.

In [ ]:
train_content,test_content=train_test_split(summaries_train,test_size=0.4,random_state=42)
train_content=train_content.drop(['student_id', 'prompt_id', 'text'],axis=1)
test_content=test_content.drop(['student_id', 'prompt_id', 'text'],axis=1)

Predicting on test data

In [ ]:
#model_names=['albert-large-v2','microsoft/deberta-v3-base','distilbert-base-uncased','bert-base-uncased','bert-base-multilingual-uncased','microsoft/MiniLM-L12-H384-uncased','microsoft/xtremedistil-l6-h256-uncased']

In [ ]:
#mcrmse_all=[]
#avg_error_all=[];
#for model_name in model_names:
#  tokenizer1 = AutoTokenizer.from_pretrained(model_name)
#  trainers=huggingface_model_train(model_name,3)
#  content_pred,wording_pred=huggingface_model_predict(test1,trainers)
#  mcrmse_all.append([mean_squared_error(content_pred, test_content['content'], squared=False),mean_squared_error(wording_pred, test_content['wording'], squared=False),np.mean([mean_squared_error(content_pred, test_content['content'], squared=False),mean_squared_error(wording_pred, test_content['wording'], squared=False)])])
#  avg_error_all.append([avg_error_cal(content_pred, test_content['content']),avg_error_cal(wording_pred, test_content['wording'])])

In [ ]:
#print(avg_error_all)

# training

In [ ]:
models=['microsoft/deberta-v3-base']

In [ ]:
torch.cuda.empty_cache()

In [ ]:
training_folds=4

In [ ]:
all_trainers_content=[]
all_trainers_wording=[]
for model_name in models:
  tokenizer1 = AutoTokenizer.from_pretrained(model_name)
  trainers=huggingface_model_train(model_name,train_content,training_folds,'content')
  all_trainers_content.append(trainers)
  trainers=huggingface_model_train(model_name,train_content,training_folds,'wording')
  all_trainers_wording.append(trainers)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fold 1


Map:   0%|          | 0/3224 [00:00<?, ? examples/s]

Map:   0%|          | 0/1075 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Fold 4


Map:   0%|          | 0/3225 [00:00<?, ? examples/s]

Map:   0%|          | 0/1074 [00:00<?, ? examples/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Rmse
1,0.531700,0.417166,0.645884
2,0.321600,0.331924,0.576129
3,0.230500,0.335284,0.579037
4,0.172600,0.414565,0.643867
5,0.127500,0.333245,0.577274


In [ ]:
LGB_model_content=LGB_train(all_trainers_content,all_trainers_wording,train_content,'content')
LGB_model_wording=LGB_train(all_trainers_content,all_trainers_wording,train_content,'wording')

# test

In [ ]:
content_preds=[]
wording_preds=[]
for trainers in all_trainers:
  content_pred=huggingface_model_predict(test_content,trainers,'content')
  wording_pred=huggingface_model_predict(test_content,trainers,'wording')
  content_preds.append(content_pred)
  wording_preds.append(wording_pred)


In [ ]:
final_content_pred,final_wording_pred=LGB_test(test_content,content_preds,wording_preds,LGB_model_content,LGB_model_wording)

In [ ]:
print(mean_squared_error(final_content_pred, test_content['content'], squared=False))
print(mean_squared_error(final_wording_pred, test_content['wording'], squared=False))
